<a href="https://colab.research.google.com/github/chandrakiranck/fuzzy-adventure/blob/master/loan_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

import os

# hide warnings
import warnings
warnings.filterwarnings('ignore')

In [0]:
loan=pd.read_csv("loan_data_train.csv")
loan.head(2)

,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,79542.0,25000,25000,18.49%,60 months,debt_consolidation,27.56%,VA,MORTGAGE,8606.56,720-724,11,15210,3.0,5 years
1,75473.0,19750,19750,17.27%,60 months,debt_consolidation,13.39%,NY,MORTGAGE,6737.50,710-714,14,19070,3.0,4 years


In [0]:
loan_tt=pd.read_csv("loan_data_test.csv")
loan_tt.head(2)

,ID,Amount.Requested,Amount.Funded.By.Investors,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,20093,5000,5000,60 months,moving,12.59%,NY,RENT,4416.67,690-694,13,7686,0,< 1 year
1,62445,18000,18000,60 months,debt_consolidation,4.93%,CA,RENT,5258.50,710-714,6,11596,0,10+ years


In [0]:
#loan=loan[loan["Employment.Length"].notnull()]
#loan_tt=loan_tt[loan_tt["Employment.Length"].notnull()]

In [0]:
loan_tt["Interest.Rate"]=np.NaN

In [0]:
loan_tt.head()

,ID,Amount.Requested,Amount.Funded.By.Investors,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length,Interest.Rate
0,20093,5000,5000,60 months,moving,12.59%,NY,RENT,4416.67,690-694,13,7686,0,< 1 year,NaN
1,62445,18000,18000,60 months,debt_consolidation,4.93%,CA,RENT,5258.50,710-714,6,11596,0,10+ years,NaN
2,65248,7200,7200,60 months,debt_consolidation,25.16%,LA,MORTGAGE,3750.00,750-754,13,7283,0,6 years,NaN
3,81822,7200,7200,36 months,debt_consolidation,17.27%,NY,MORTGAGE,3416.67,790-794,14,4838,0,10+ years,NaN
4,57923,22000,22000,60 months,debt_consolidation,18.28%,MI,MORTGAGE,6083.33,720-724,9,20181,0,8 years,NaN


In [0]:
loan["data"]="Train"
loan_tt["data"]="Test"

#### ordering the columns as like different dataset(train)

In [0]:
loan_fl=pd.concat([loan,loan_tt],axis=0)

In [0]:
loan_fl.tail(2)

,Amount.Funded.By.Investors,Amount.Requested,Debt.To.Income.Ratio,Employment.Length,FICO.Range,Home.Ownership,ID,Inquiries.in.the.Last.6.Months,Interest.Rate,Loan.Length,Loan.Purpose,Monthly.Income,Open.CREDIT.Lines,Revolving.CREDIT.Balance,State,data
298,13600,13600,12.85%,10+ years,675-679,RENT,36931.0,0.0,NaN,60 months,credit_card,5651.33,13,24383,MN,Test
299,6000,6000,7.64%,2 years,675-679,MORTGAGE,34744.0,0.0,NaN,36 months,debt_consolidation,5184.75,8,14422,IL,Test


In [0]:
loan_fl.shape

(2500, 16)

In [0]:
loan_fl.dtypes

Amount.Funded.By.Investors         object
Amount.Requested                   object
Debt.To.Income.Ratio               object
Employment.Length                  object
FICO.Range                         object
Home.Ownership                     object
ID                                float64
Inquiries.in.the.Last.6.Months    float64
Interest.Rate                      object
Loan.Length                        object
Loan.Purpose                       object
Monthly.Income                    float64
Open.CREDIT.Lines                  object
Revolving.CREDIT.Balance           object
State                              object
data                               object
dtype: object

In [0]:
loan_fl.drop(["ID","Amount.Funded.By.Investors"],axis=1,inplace=True)

In [0]:
loan_fl

,Amount.Requested,Debt.To.Income.Ratio,Employment.Length,FICO.Range,Home.Ownership,Inquiries.in.the.Last.6.Months,Interest.Rate,Loan.Length,Loan.Purpose,Monthly.Income,Open.CREDIT.Lines,Revolving.CREDIT.Balance,State,data
0,25000,27.56%,5 years,720-724,MORTGAGE,3.0,18.49%,60 months,debt_consolidation,8606.56,11,15210,VA,Train
1,19750,13.39%,4 years,710-714,MORTGAGE,3.0,17.27%,60 months,debt_consolidation,6737.50,14,19070,NY,Train
2,2100,3.50%,< 1 year,690-694,OWN,1.0,14.33%,36 months,major_purchase,1000.00,13,893,LA,Train
3,28000,19.62%,10+ years,710-714,MORTGAGE,1.0,16.29%,36 months,credit_card,7083.33,12,38194,NV,Train
4,24250,23.79%,10+ years,730-734,MORTGAGE,2.0,12.23%,60 months,credit_card,5833.33,6,31061,OH,Train
5,5400,6.27%,NaN,785-789,RENT,2.0,8.90%,36 months,small_business,3000.00,2,90,OH,Train
6,16000,11.27%,2 years,665-669,RENT,0.0,22.47%,60 months,credit_card,4791.67,5,8474,CA,Train
7,6400,9.58%,< 1 year,735-739,RENT,1.0,7.51%,36 months,major_purchase,6400.00,11,12306,IL,Train
8,24000,15.54%,NaN,740-744,MORTGAGE,0.0,12.99%,60 months,home_improvement,8333.33,24,56984,NC,Train
9,8000,0%,NaN,765-769,MORTGAGE,1.0,7.62%,36 months,other,3500.00,7,0,NC,Train


In [0]:


for col in ["Debt.To.Income.Ratio","Interest.Rate"]:
    loan_fl[col]=loan_fl[col].str.replace("%"," ")

In [0]:
loan_fl

,Amount.Requested,Debt.To.Income.Ratio,Employment.Length,FICO.Range,Home.Ownership,Inquiries.in.the.Last.6.Months,Interest.Rate,Loan.Length,Loan.Purpose,Monthly.Income,Open.CREDIT.Lines,Revolving.CREDIT.Balance,State,data
0,25000,27.56,5 years,720-724,MORTGAGE,3.0,18.49,60 months,debt_consolidation,8606.56,11,15210,VA,Train
1,19750,13.39,4 years,710-714,MORTGAGE,3.0,17.27,60 months,debt_consolidation,6737.50,14,19070,NY,Train
2,2100,3.50,< 1 year,690-694,OWN,1.0,14.33,36 months,major_purchase,1000.00,13,893,LA,Train
3,28000,19.62,10+ years,710-714,MORTGAGE,1.0,16.29,36 months,credit_card,7083.33,12,38194,NV,Train
4,24250,23.79,10+ years,730-734,MORTGAGE,2.0,12.23,60 months,credit_card,5833.33,6,31061,OH,Train
5,5400,6.27,NaN,785-789,RENT,2.0,8.90,36 months,small_business,3000.00,2,90,OH,Train
6,16000,11.27,2 years,665-669,RENT,0.0,22.47,60 months,credit_card,4791.67,5,8474,CA,Train
7,6400,9.58,< 1 year,735-739,RENT,1.0,7.51,36 months,major_purchase,6400.00,11,12306,IL,Train
8,24000,15.54,NaN,740-744,MORTGAGE,0.0,12.99,60 months,home_improvement,8333.33,24,56984,NC,Train
9,8000,0,NaN,765-769,MORTGAGE,1.0,7.62,36 months,other,3500.00,7,0,NC,Train


In [0]:
for col in ["Amount.Requested","Interest.Rate","Debt.To.Income.Ratio","Open.CREDIT.Lines","Revolving.CREDIT.Balance"]:
    loan_fl[col]=pd.to_numeric(loan_fl[col],errors="coerce")

In [0]:
loan_fl.dtypes

Amount.Requested                  float64
Debt.To.Income.Ratio              float64
Employment.Length                  object
FICO.Range                         object
Home.Ownership                     object
Inquiries.in.the.Last.6.Months    float64
Interest.Rate                     float64
Loan.Length                        object
Loan.Purpose                       object
Monthly.Income                    float64
Open.CREDIT.Lines                 float64
Revolving.CREDIT.Balance          float64
State                              object
data                               object
dtype: object

In [0]:
loan_fl["FICO1"]=loan_fl["FICO.Range"].str.split("-", n=1, expand=True)[0]
loan_fl["FICO1"].head(2)

0    720
1    710
Name: FICO1, dtype: object

In [0]:
loan_fl["FICO2"]=loan_fl["FICO.Range"].str.split("-", n=1, expand=True)[1]

In [0]:
loan_fl["FICO2"].head(2)

0    724
1    714
Name: FICO2, dtype: object

In [0]:
loan_fl.drop(["FICO.Range"],axis=1,inplace=True)

In [0]:
loan_fl.head(2)

,Amount.Requested,Debt.To.Income.Ratio,Employment.Length,Home.Ownership,Inquiries.in.the.Last.6.Months,Interest.Rate,Loan.Length,Loan.Purpose,Monthly.Income,Open.CREDIT.Lines,Revolving.CREDIT.Balance,State,data,FICO1,FICO2
0,25000.0,27.56,5 years,MORTGAGE,3.0,18.49,60 months,debt_consolidation,8606.56,11.0,15210.0,VA,Train,720,724
1,19750.0,13.39,4 years,MORTGAGE,3.0,17.27,60 months,debt_consolidation,6737.50,14.0,19070.0,NY,Train,710,714


In [0]:
for col in ["FICO1","FICO2"]:
    loan_fl[col]=pd.to_numeric(loan_fl[col],errors="coerce")

In [0]:
loan_fl["FICO.Range"]=0.5*(loan_fl['FICO1'] + loan_fl['FICO2'])

In [0]:
loan_fl.drop(["FICO1","FICO2"],axis=1,inplace=True)

In [0]:
loan_fl.head()

,Amount.Requested,Debt.To.Income.Ratio,Employment.Length,Home.Ownership,Inquiries.in.the.Last.6.Months,Interest.Rate,Loan.Length,Loan.Purpose,Monthly.Income,Open.CREDIT.Lines,Revolving.CREDIT.Balance,State,data,FICO.Range
0,25000.0,27.56,5 years,MORTGAGE,3.0,18.49,60 months,debt_consolidation,8606.56,11.0,15210.0,VA,Train,722.0
1,19750.0,13.39,4 years,MORTGAGE,3.0,17.27,60 months,debt_consolidation,6737.50,14.0,19070.0,NY,Train,712.0
2,2100.0,3.50,< 1 year,OWN,1.0,14.33,36 months,major_purchase,1000.00,13.0,893.0,LA,Train,692.0
3,28000.0,19.62,10+ years,MORTGAGE,1.0,16.29,36 months,credit_card,7083.33,12.0,38194.0,NV,Train,712.0
4,24250.0,23.79,10+ years,MORTGAGE,2.0,12.23,60 months,credit_card,5833.33,6.0,31061.0,OH,Train,732.0


In [0]:
loan_fl["Employment.Length"]=loan_fl["Employment.Length"].str.replace("years","")

In [0]:
loan_fl["Employment.Length"]=loan_fl["Employment.Length"].str.replace("year","")

In [0]:
loan_fl["Employment.Length"]=np.where(loan_fl["Employment.Length"].str[:2]=="10",10,loan_fl["Employment.Length"])

In [0]:
loan_fl["Employment.Length"]=np.where(loan_fl["Employment.Length"].str[0]=="<",0,loan_fl["Employment.Length"])

In [0]:
loan_fl["Employment.Length"]=pd.to_numeric(loan_fl["Employment.Length"],errors="coerce")

In [0]:
loan_fl["Employment.Length"].value_counts()

10.0    653
0.0     249
2.0     243
3.0     235
5.0     202
4.0     191
1.0     177
6.0     163
7.0     127
8.0     108
9.0      72
Name: Employment.Length, dtype: int64

In [0]:
loan_fl.head(10)

NameError: name 'loan_fl' is not defined

In [0]:
cat_cols=loan_fl.select_dtypes(["object"]).columns

In [0]:
cat_cols

Index(['Home.Ownership', 'Loan.Length', 'Loan.Purpose', 'State', 'data'], dtype='object')

In [0]:
cat_cols=cat_cols[:-1]

In [0]:
for col in cat_cols:
    freqs=loan_fl[col].value_counts()
    k=freqs.index[freqs>20][:-1]
    for cat in k:
        name=col+"_"+cat
        loan_fl[name]=(loan_fl[col]==cat).astype(int)
    del loan_fl[col]
    print(col)

Home.Ownership
Loan.Length
Loan.Purpose
State


In [0]:
loan_fl.shape

(2500, 51)

In [0]:
loan_fl.isnull().sum()

Amount.Requested                     5
Debt.To.Income.Ratio                 1
Employment.Length                   80
Inquiries.in.the.Last.6.Months       3
Interest.Rate                      300
Monthly.Income                       3
Open.CREDIT.Lines                    9
Revolving.CREDIT.Balance             5
data                                 0
FICO.Range                           0
Home.Ownership_MORTGAGE              0
Home.Ownership_RENT                  0
Loan.Length_36 months                0
Loan.Purpose_debt_consolidation      0
Loan.Purpose_credit_card             0
Loan.Purpose_other                   0
Loan.Purpose_home_improvement        0
Loan.Purpose_major_purchase          0
Loan.Purpose_small_business          0
Loan.Purpose_car                     0
Loan.Purpose_wedding                 0
Loan.Purpose_medical                 0
Loan.Purpose_moving                  0
State_CA                             0
State_NY                             0
State_TX                 

In [0]:
for col in loan_fl.columns:
    if(col not in ["Interest.Rate","data"])&(loan_fl[col].isnull().sum()>0):
        loan_fl.loc[loan_fl[col].isnull(),col]=loan_fl.loc[loan_fl["data"]=="Train",col].mean()

In [0]:
loan_fl.isnull().sum()

Amount.Requested                     0
Debt.To.Income.Ratio                 0
Employment.Length                    0
Inquiries.in.the.Last.6.Months       0
Interest.Rate                      300
Monthly.Income                       0
Open.CREDIT.Lines                    0
Revolving.CREDIT.Balance             0
data                                 0
FICO.Range                           0
Home.Ownership_MORTGAGE              0
Home.Ownership_RENT                  0
Loan.Length_36 months                0
Loan.Purpose_debt_consolidation      0
Loan.Purpose_credit_card             0
Loan.Purpose_other                   0
Loan.Purpose_home_improvement        0
Loan.Purpose_major_purchase          0
Loan.Purpose_small_business          0
Loan.Purpose_car                     0
Loan.Purpose_wedding                 0
Loan.Purpose_medical                 0
Loan.Purpose_moving                  0
State_CA                             0
State_NY                             0
State_TX                 

In [0]:
loan_train=loan_fl[loan_fl["data"]=="Train"]
del  loan_train["data"]

In [0]:
loan_test=loan_fl[loan_fl["data"]=="Test"]
loan_test.drop(["Interest.Rate","data"],axis=1,inplace=True)

In [0]:
loan_train.shape

(2200, 50)

In [0]:
loan_test.shape

(300, 49)

In [0]:
del loan_fl

In [0]:
from sklearn.model_selection import train_test_split
loan_train1,loan_train2=train_test_split(loan_train,test_size=0.20,random_state =2)


In [0]:
x_train1=loan_train1.drop("Interest.Rate",axis=1)
y_train1=loan_train1["Interest.Rate"]

In [0]:
from sklearn.linear_model import LinearRegression

In [0]:
lm=LinearRegression()

In [0]:
lm.fit(x_train1,y_train1)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
lm.intercept_

76.01913622061615

In [0]:
list(zip(x_train1.columns,lm.coef_))

[('Amount.Requested', 0.00015614431012152515),
 ('Debt.To.Income.Ratio', -0.0038442407744355357),
 ('Employment.Length', 0.03473388302290284),
 ('Inquiries.in.the.Last.6.Months', 0.3367613029580375),
 ('Monthly.Income', -2.6406486110575056e-05),
 ('Open.CREDIT.Lines', -0.04022799798491161),
 ('Revolving.CREDIT.Balance', -4.043165672484861e-06),
 ('FICO.Range', -0.08669304194246436),
 ('Home.Ownership_MORTGAGE', -0.49168102215386755),
 ('Home.Ownership_RENT', -0.23217518504343101),
 ('Loan.Length_36 months', -3.142554743125106),
 ('Loan.Purpose_debt_consolidation', -0.47135975236196836),
 ('Loan.Purpose_credit_card', -0.6123429134595983),
 ('Loan.Purpose_other', 0.44626098118804486),
 ('Loan.Purpose_home_improvement', -0.36998297809737823),
 ('Loan.Purpose_major_purchase', -0.09745119837232585),
 ('Loan.Purpose_small_business', 0.06251542143810068),
 ('Loan.Purpose_car', 0.01874891431692033),
 ('Loan.Purpose_wedding', -0.7830520540035637),
 ('Loan.Purpose_medical', -0.42214589469285724)

In [0]:
x_train2=loan_train2.drop("Interest.Rate",axis=1)

In [0]:
predicted_ir=lm.predict(x_train2)

In [0]:
from sklearn.metrics import mean_absolute_error

In [0]:
mean_absolute_error(loan_train2["Interest.Rate"],predicted_ir)

1.629229219461918

In [0]:
x_train=loan_train.drop("Interest.Rate",axis=1)
y_train=loan_train["Interest.Rate"]

In [0]:
lm.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
test_predict=lm.predict(loan_test)

In [0]:
pd.DataFrame(test_predict).to_csv("predicted.csv",index=False)

## Ridge regression

In [0]:
rr=Ridge(alpha=0.01)
#restricted and in this linear and ridge regression resembles
rr.fit(x_train,y_train)

Ridge(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [0]:
train_score_rr=rr.score(x_train,y_train)
print(train_score_rr)



0.764174771770492


In [0]:
rr100=Ridge(alpha=100)
#restricted and in this linear and ridge regression resembles
rr100.fit(x_train,y_train)
train_score_rr100=rr100.score(x_train,y_train)
print(train_score_rr100)

0.7568709581047769


In [0]:
rr10000=Ridge(alpha=10000)
#restricted and in this linear and ridge regression resembles
rr10000.fit(x_train,y_train)
train_score_rr10000=rr10000.score(x_train,y_train)
print(train_score_rr10000)

0.6712740291490613


In [0]:
lambdas=np.linspace(1,100,100)

In [0]:
params={"alpha":lambdas}

In [0]:
model=Ridge(fit_intercept=True)

In [0]:
grid_search=GridSearchCV(model,param_grid=params,cv=10)

In [0]:
grid_search.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,...
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100.]

In [0]:
grid_search.best_estimator_

Ridge(alpha=24.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [0]:
test_predict=grid_search.predict(loan_test)

In [0]:
ridge_model=grid_search.best_estimator_

In [0]:
ridge_model.fit(x_train,y_train)

Ridge(alpha=24.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [0]:
list(zip(x_train.columns,ridge_model.coef_))

[('Amount.Requested', 0.0001652095878186319),
 ('Debt.To.Income.Ratio', -0.0012607895541777952),
 ('Employment.Length', 0.019749889686731067),
 ('Inquiries.in.the.Last.6.Months', 0.34207202870931547),
 ('Monthly.Income', -2.8738738226306198e-05),
 ('Open.CREDIT.Lines', -0.03648051418534769),
 ('Revolving.CREDIT.Balance', -2.969978145994554e-06),
 ('FICO.Range', -0.08662094002097392),
 ('Home.Ownership_MORTGAGE', -0.32521643871476363),
 ('Home.Ownership_RENT', -0.11642196883137129),
 ('Loan.Length_36 months', -2.917688808430892),
 ('Loan.Purpose_debt_consolidation', -0.39801668156794856),
 ('Loan.Purpose_credit_card', -0.4939984021491051),
 ('Loan.Purpose_other', 0.37260225004629377),
 ('Loan.Purpose_home_improvement', -0.2709538502430429),
 ('Loan.Purpose_major_purchase', -0.03656795735440937),
 ('Loan.Purpose_small_business', 0.07372450258357965),
 ('Loan.Purpose_car', -0.06374668371795089),
 ('Loan.Purpose_wedding', -0.3943903892366554),
 ('Loan.Purpose_medical', -0.12557136802090854

In [0]:
lasso=Lasso()
lasso_model=lasso.fit(x_train,y_train)
train_score=lasso.score(x_train,y_train)

coeff_used=np.sum(lasso.coef_!=0)

print('training score'),train_score


training score


(None, 0.6584358102060043)

In [0]:
print(lasso_model.coef_)

[ 2.23628338e-04 -0.00000000e+00  0.00000000e+00  0.00000000e+00
 -5.05390887e-05 -0.00000000e+00 -8.70817718e-06 -8.64676523e-02
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00]


In [0]:
print("number of features used:",),coeff_used

number of features used:


(None, 4)

In [0]:
list(zip(x_train.columns,lasso_model.coef_))

[('Amount.Requested', 0.00022362833779463023),
 ('Debt.To.Income.Ratio', -0.0),
 ('Employment.Length', 0.0),
 ('Inquiries.in.the.Last.6.Months', 0.0),
 ('Monthly.Income', -5.053908873061881e-05),
 ('Open.CREDIT.Lines', -0.0),
 ('Revolving.CREDIT.Balance', -8.708177177588524e-06),
 ('FICO.Range', -0.08646765227679754),
 ('Home.Ownership_MORTGAGE', -0.0),
 ('Home.Ownership_RENT', 0.0),
 ('Loan.Length_36 months', -0.0),
 ('Loan.Purpose_debt_consolidation', -0.0),
 ('Loan.Purpose_credit_card', -0.0),
 ('Loan.Purpose_other', 0.0),
 ('Loan.Purpose_home_improvement', 0.0),
 ('Loan.Purpose_major_purchase', 0.0),
 ('Loan.Purpose_small_business', 0.0),
 ('Loan.Purpose_car', 0.0),
 ('Loan.Purpose_wedding', -0.0),
 ('Loan.Purpose_medical', -0.0),
 ('Loan.Purpose_moving', 0.0),
 ('State_CA', -0.0),
 ('State_NY', 0.0),
 ('State_TX', 0.0),
 ('State_FL', -0.0),
 ('State_IL', -0.0),
 ('State_GA', -0.0),
 ('State_PA', -0.0),
 ('State_NJ', -0.0),
 ('State_VA', 0.0),
 ('State_MA', -0.0),
 ('State_OH', -0.

In [0]:
lasso001=Lasso(alpha=0.01,max_iter=10000)
lasso001.fit(x_train,y_train)
train_score_001=lasso001.score(x_train,y_train)

print(train_score_001)

0.7607472742910711


In [0]:
lambdas=np.linspace(0.001,2,100)
model=Lasso(fit_intercept=True)
params={"alpha":lambdas}

In [0]:
grid_search=GridSearchCV(model,param_grid=params,cv=10)

In [0]:
grid_search.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([1.00000000e-03, 2.11919192e-02, 4.13838384e-02, 6.15757576e-02,
       8.17...
       1.61635354e+00, 1.63654545e+00, 1.65673737e+00, 1.67692929e+00,
       1.69712121e+00, 1.71731313e+00, 1.73750505e+00, 1.75769697e+00,
       1.77788889e+00, 1.79808081e+00, 1.81827273e+00, 1.83846465e+00,
       1.85865657e+00, 1.87884848e+00, 1.89904040e+00, 1.91923232e+00,
       1.93942424e+00, 1.95961616e+00, 1.97980808e+00, 2.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [0]:
grid_search.best_estimator_

Lasso(alpha=0.021191919191919192, copy_X=True, fit_intercept=True,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [0]:
from pandas.plotting import lag_plot

lag_plot(loan_fl)

NameError: name 'loan_fl' is not defined